In [1]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

# Add here the path of your downloaded open-source model. Here I'm using "llama-2-7b-chat.ggmlv3.q2_K.bin" from HuggingFace.
MODEL_PATH = "llama-2-7b-chat.ggmlv3.q2_K.bin" 

DB_FAISS_PATH = 'vectorstore/test_db_faiss'


custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vector stores.
    """

    prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context','question'])

    return prompt


#Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain



#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = MODEL_PATH,
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )
    return llm

#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cuda'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

#output function
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response



/Users/abidsaudagar/Personal/AI-chatbot-chat-with-PDFs-using-open-source-models-works-offline-as-well/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
q1 = final_result('why building habit is difficult?')

In [8]:
q1['result']

"Does it have something to do with procrastination, lack of motivation or focus, or a mix of all of these?\n\nI don't know. "

In [9]:
q1["source_documents"]

[Document(page_content='Why Is It So Hard to Stick to Good \nHabits? \n \n \nIt seems to be remarkably easy to fall into unhealthy routines.  \n \n★Eating junk food.  \n★Watching TV instead of going to the gym.  \n★Showing up to a job you hate everyday.  \n★Biting your nails. \n★Smoking.  \n \nThere’s no shortage of unhealthy and unproductive behaviors. And we all struggle \nwith them from time to time. \n \nBut why? You want to live a healthy, fulfilling, and remarkable life. And every now', metadata={'source': 'test_data/james-clear-transform-your-habits-v3.pdf', 'page': 4}),
 Document(page_content="you. It’s tough to find a reward in something when you’re only doing it because you think \nother people expect it or would approve of it. It's your life, so make sure you're spending \nyour time on things that are important to you. \n \nNow that we've covered the science of habit formation, let's talk about how to use \nit in real life.  \n \n \n \n \n \n \nJamesClear.com Page 14", metad

In [12]:
q2 = final_result("how to make it easy?")

In [13]:
q2['result']

"You don't have to try too hard to make things easy. Just start with small, incremental steps toward your goals every day. It's okay if it doesn't happen all at once. "

In [14]:
q2["source_documents"]

[Document(page_content='on life in general.  \n \n \nStep 2: Make Your Habits Incredibly Easy to Start  \n \nMake it so easy you can’t say no.  \n—Leo Babauta \n \nAs I mentioned in the beginning of this guide, it’s incredibly easy to get caught up \nin the desire to make massive changes in your life. We watch incredible weight loss \n \nJamesClear.com Page 12', metadata={'source': 'test_data/james-clear-transform-your-habits-v3.pdf', 'page': 12}),
 Document(page_content='focus on getting things done.  \n \nI’m not always on top of my game, but on the days I train everything seems to \ncome a little bit easier. Exercise naturally pushes me towards my best self.  \n \n \nWhat Are Your Keystone Habits? \n \nImproving your lifestyle and becoming the type of person who “has their act \ntogether” isn’t nearly as hard as you might think. In fact, you might need just \none keystone habit before the dominoes start falling everywhere.  \n \n \nJamesClear.com Page 31', metadata={'source': 'test_

In [15]:
q3 = final_result("who is the author?")

In [16]:
q3['result']

'The author of James Clear\'s "Making Habit Stacking Work" is James Clear himself.'

In [17]:
q3["source_documents"]

[Document(page_content='JamesClear.com Page 10', metadata={'source': 'test_data/james-clear-transform-your-habits-v3.pdf', 'page': 10}),
 Document(page_content='names.”  \n \nEven today, she is great at remembering the names of anyone we come across.  \n \nJamesClear.com Page 16', metadata={'source': 'test_data/james-clear-transform-your-habits-v3.pdf', 'page': 16})]